# Desafio - Twitter

## Desenvolvimento de crawlers

In [ ]:
#importar bibliotecas
import json
from tweepy import OAuthHandler, Stream, StreamListener
from datetime import datetime

In [ ]:
#Cadastrar chaves de acesso
consumer_key = "Tg58lI63lgbANDvkQsJin17mJ"
consumer_secret = "72TMKUWaXPdRMUm6fDcNbLEY8r0YhKBiRLcafzThZycQCojl68"

access_token = "1313174623681482753-eooKFusfY61mN14jKJEpnXR7yjxQC1"
Access_token_secret = "klGgCyQhWxT6JkqL090Nnapz2WkYtZeWlex6v3NeQ4npL"

In [ ]:
#Definir um arquivo de saída para armazenar os twtees coletados
data_hoje = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
out = open(f"collected_tweets_{data_hoje}.txt","w")

In [ ]:
#Implemmentar uma classe para conexão com o Twitter
class MyListener (StreamListener):
    
    def on_data(self, data):
        itemString = json.dumps(data)
        out.write(itemString + "\n")
        return True
    
    def on_error(self, status):
        print(status)
    

In [ ]:
#implementar a função MAIN
if __name__ == "__main__":
    l = MyListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, Access_token_secret)
    
    stream = Stream(auth, l)
    stream.filter(track=["Trump"]) #["Netflix", "HBOGO", "Globoplay", "Disney Pl"]

## Estrutura do Json

In [ ]:
import json

In [ ]:
#Abrir o arquivo de twitter e ler as linhas
with open("collected_tweets_2020-11-30-22-05-28.txt",'r') as file:
    tweets = file.readlines()

In [ ]:
#Ler a primeira linha
tweets[0]

In [ ]:
#Escrever o arquivo no formato Json, para ficar no formato dicionário precisa de converter duas vezes json.loads
with open('tweet.json', 'w') as f:
    json.dump(json.loads(json.loads(tweets[0])),f)

In [ ]:
#Escrever o arquivo em formato de lista
#Exemplo do que irá ser feito abaixo [i for i in range(10)]
parsed_tweets = [json.loads(json.loads(i)) for i in tweets ]
#Ver o tamanho dos parsed_tweets
len(parsed_tweets)

## Acessando as chaves do Json (dicionáirio)

In [ ]:
#Trabalhando com o primeiro tweet
primeiroTweet = parsed_tweets[100]

In [ ]:
#teste para pegar as chaves do primeiro nivel do Tweet
primeiroTweet['id']

In [ ]:
#Acessar a chave do segundo nível do tweet
primeiroTweet['user']['name']

In [ ]:
#Acessar a chave do terceiro nível do tweet - tweet original
primeiroTweet['retweeted_status']['entities']['user_mentions'][0]['name']

## Transformar os Tweets em estruta relacional

In [ ]:
#importar o pandas para colocar em estrutura de tabela
import pandas as pd

In [ ]:
#Transforma em dataFrame com os campos somente os dados nas linhas
df_tratado = pd.DataFrame(primeiroTweet).reset_index(drop=True).iloc[:1]#pega somente a primeira linha até 1 exclusivo
df_tratado

In [ ]:
df_tratado.columns

In [ ]:
#eliminar as colunas
df_tratado.drop(columns = ['quote_count','reply_count', 'retweet_count', 'favorite_count',
                          'favorited', 'retweeted', 'user', 'entities', 'retweeted_status' ], inplace = True)

In [ ]:
df_tratado.columns

In [ ]:
#Criando novas colunas segundo nivel e atribuindo valores
df_tratado['user_id'] = primeiroTweet['user']['id']
df_tratado['user_id_str'] = primeiroTweet['user']['id_str']
df_tratado['user_screen_name'] = primeiroTweet['user']['screen_name']
df_tratado['user_location'] = primeiroTweet['user']['location']

In [ ]:
df_tratado.columns

In [ ]:
# trabalhar com segundo e terceiro níveis
user_mentions = []
for i in range(len(primeiroTweet['entities']['user_mentions'])):
    dicionariobase = primeiroTweet['entities']['user_mentions'][i].copy()
    dicionariobase.pop('indices', None)
    df = pd.DataFrame(dicionariobase, index = [0])
    df = df.rename(columns = {
        'screem_mame': 'entities_screen_name',
        'name': 'entities_name',
        'id': 'entities_id',
        'id_str': 'entities_id-str'
    })
    
    user_mentions.append(df)

In [ ]:
user_mentions[0]

In [ ]:
#empilhar e concatenar com a primeira tabela
dfs = []
for i in user_mentions:
    dfs.append(
        pd.concat([df_tratado.copy(), i], axis = 1))

pd.concat(dfs, ignore_index = True)

## Função que faz todo o tratamento do tweet e convert par aum dataFrame Pandas

In [ ]:
def tweet_para_df (tweet):
    try:
        df_tratado = pd.DataFrame(tweet).reset_index(drop=True).iloc[:1]
        df_tratado.drop(columns = ['quote_count','reply_count', 'retweet_count', 'favorite_count',
                              'favorited', 'retweeted', 'user', 'entities', 'retweeted_status' ], inplace = True)

        df_tratado['user_id'] = tweet['user']['id']
        df_tratado['user_id_str'] = tweet['user']['id_str']
        df_tratado['user_screen_name'] = tweet['user']['screen_name']
        df_tratado['user_location'] = tweet['user']['location']

        user_mentions = []

        for i in range(len(tweet['entities']['user_mentions'])):
            dicionariobase = tweet['entities']['user_mentions'][i].copy()
            dicionariobase.pop('indices', None)
            df = pd.DataFrame(dicionariobase, index = [0])
            df = df.rename(columns = {
                'screem_mame': 'entities_screen_name',
                'name': 'entities_name',
                'id': 'entities_id',
                'id_str': 'entities_id-str'
            })

            user_mentions.append(df)

        #empilhar e concatenar com a primeira tabela
        dfs = []
        for i in user_mentions:
            dfs.append(
                pd.concat([df_tratado.copy(), i], axis = 1))

        df_final = pd.concat(dfs, ignore_index = True)
    except:
        return None
        
    return df_final

In [ ]:
#Testes para ver se funciona a função
tweet_para_df (parsed_tweets[100])

In [ ]:
 %%time
#conta o tempo de execução
# Iteração e transformação todos os twitter

parseados = [tweet_para_df (tweet) for tweet in parsed_tweets]

In [ ]:
#Eliminar os valores vazios
parseados = [i for i in parseados if i is not None]


In [ ]:
len(parseados)

In [ ]:
#concatenando as linhas na vertical
tratado = pd.concat(parseados, ignore_index = True)


In [ ]:
#Teste
len(tratado)

## Importando os dados para o banco MySQL

In [ ]:
#importa bliblioteca para conectar em MySql
import pyodbc
import mysql.connector
# Import dataframe into MySQL
import sqlalchemy

In [ ]:
username = 'root'
password = 'igti'
ip       = 'localhost'
database_name = 'bootcamp'
con = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(username, password,ip, database_name))

In [ ]:
#ingestão dos dados do Twitter Trampp

tratado.to_sql(con=con, name='twitterTrampp', if_exists='replace',index=False )